In [ ]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import os
from typing import Dict
from IPython.display import display, Markdown

In [ ]:
load_dotenv(override=True)

In [ ]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
search_iteration = 3

search_instructions = f"You are a very helpful research assistant. \
                        Given a query, come up with a set of searches to perform the best regarding the answer to the query. \
                        Output {search_iteration} terms to query for."

class WebSearchItem(BaseModel):
    reason: str = Field(description='Your reasoning for why this search is important to the query.')
    query: str = Field(description='The search term to use for the web search.')

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description='A list of web searches to best answer the query.')

planner_agent = Agent(
    name='PlannerAgent',
    instructions=search_instructions,
    model='gpt-4o-mini',
    output_type=WebSearchPlan
)

In [ ]:
message = 'Which penny stock can be invested upon at this time in Indian market'

with trace('Search'):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

In [ ]:
report_instructions = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary: str = Field(description='A short 2-3 sentence summary of the findings')
    markdown_report: str = Field(description='The final report')
    follow_up_questions: list[str] = Field(description='Suggested topics to research further')

writer_agent = Agent(
    name='ReportWriterAgent',
    instructions=report_instructions,
    model='gpt-4o-mini',
    output_type=ReportData
)

In [ ]:
@function_tool
def save_to_html_file(content: str, filename: str = "MarketResearchReport.html"):
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(content)
        print(f"Content successfully written to {filename}")
    except Exception as e:
        print(f"An error occurred while writing to the file: {e}")

In [ ]:
save_report_instructions = """You are able to save a detailed report in a nicely formatted way.
    You will be provided with a detailed report. You should use your tool to save the report, 
    providing the report converted into clean, well presented HTML with appropriate subject header."""

save_report_agent = Agent(
    name='SaveReportAgent',
    instructions=save_report_instructions,
    tools=[save_to_html_file],
    model='gpt-4o-mini'
)

In [ ]:
async def plan_searches(query: str):
    print('Planning searches...')
    result = await Runner.run(planner_agent, f'query: {query}')
    print(f'Will perform {len(result.final_output.searches)} searches')
    return result.final_output

async def perform_searches(searchPlan: WebSearchPlan):
    print('Searching...')
    tasks = [asyncio.create_task(search(item)) for item in searchPlan.searches]
    results = await asyncio.gather(*tasks)
    print('Finished Searching')
    return results

async def search(item: WebSearchItem):
    input = 'Search term: {item.query}\n Reason for searching: {item.reason}'
    result = await Runner.run(search_agent, input)
    return result.final_output

async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def save_report_to_file(report: ReportData):
    print('Saving report...')
    result = await Runner.run(save_report_agent, report.markdown_report)
    print('Report created')
    return report

In [ ]:
query = 'Which penny stock can be invested upon at this time in Indian market'

with trace('Research trace'):
    print('Starting research...')
    search_plan = await plan_searches(query)
    search_result = await perform_searches(search_plan)
    report = await write_report(query, search_result)
    await save_report_to_file(report)
    print('Report Created')